In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import talib as ta

In [15]:
activo = 'ETH-USD'
df = yf.Ticker(activo)
df = df.history(start='2018-01-01', end='2024-10-01')

In [16]:
def backtest_strategy(buy_threshold, sell_threshold):
    df['MACD'], df['Signal_line'], df['Histogram'] = ta.MACD(df['Close'], fastperiod=12, 
                                                        slowperiod=26, signalperiod=9)
    df['Signal'] = np.where(
        (df['MACD'] > df['Signal_line']) & (df['Histogram'] > buy_threshold) ,1,
        np.where(
            (df['MACD'] < df['Signal_line']) & (df['Histogram'] < sell_threshold), -1, 0
        )
    )
    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    return (df['Strategy_Returns'] + 1).cumprod().iloc[-1] -1 # Retorno total

In [17]:
# Definir el rango de parámetros a probar
buy_thresholds = range(-30, 0, 1)
sell_thresholds = range(0, 30, 1)

In [18]:
# Probar todas las combinaciones de parámetros
best_performance = -np.inf
best_params = (None, None)

In [23]:
for buy_threshold in buy_thresholds:
    for sell_threshold in sell_thresholds:
        if buy_threshold >= sell_threshold:
            continue
        performance = backtest_strategy(buy_threshold, sell_threshold)
        if performance > best_performance:
            best_performance = performance
            best_params = (buy_threshold, sell_threshold)

print(f"Mejores parámetros: Buy Threshold={best_params[0]}, sell Threshold={best_params[1]}")
print(f'Mejor Rendimiento: {best_performance:.2%}')

Mejores parámetros: Buy Threshold=-30, sell Threshold=0
Mejor Rendimiento: 375.94%
